In [1]:
!pip install segment-geospatial

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  

In [2]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from samgeo.hq_sam import SamGeo
from samgeo.common import array_to_image

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.10/dist-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_5m_224 in registry with segment_anything_hq.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarnin

In [3]:
sam_kwargs = {
    "points_per_side": 32,
    "pred_iou_thresh": 0.8,
    "stability_score_thresh": 0.9,
    "crop_n_layers": 1,
    "crop_n_points_downscale_factor": 2,
    "min_mask_region_area": 50,
}

sam = SamGeo(
    model_type="vit_h",
    sam_kwargs=sam_kwargs,
)

Model checkpoint for vit_h not found.


Downloading...
From: https://github.com/opengeos/datasets/releases/download/models/sam_hq_vit_h.zip
To: /root/.cache/torch/hub/checkpoints/sam_hq_vit_h.zip
100%|██████████| 1.02G/1.02G [00:10<00:00, 93.4MB/s]
Downloading...
From: https://github.com/opengeos/datasets/releases/download/models/sam_hq_vit_h.z01
To: /root/.cache/torch/hub/checkpoints/sam_hq_vit_h.z01
100%|██████████| 1.36G/1.36G [00:14<00:00, 96.4MB/s]
INFO patool: Extracting /root/.cache/torch/hub/checkpoints/sam_hq_vit_h.zip ...
INFO patool: running /usr/bin/7z x -aou -o/root/.cache/torch/hub/checkpoints -- /root/.cache/torch/hub/checkpoints/sam_hq_vit_h.zip
INFO patool: ... /root/.cache/torch/hub/checkpoints/sam_hq_vit_h.zip extracted to `/root/.cache/torch/hub/checkpoints'.
/usr/local/lib/python3.10/dist-packages/segment_anything_hq/build_sam.py:160: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construc

<All keys matched successfully>


In [4]:
output_dir = 'segs'
os.mkdir(output_dir)

In [5]:
dirname = "/kaggle/input/sentinel2/img1/tiles"

for filename in sorted(os.listdir(dirname)):
    tile_name = filename.split('.')[0]
    img_path = os.path.join(dirname, filename)
    
    sam.generate(img_path, foreground=True, unique=True)
    
    sam.show_anns(axis="off", alpha=1, blend=False)
    plt.close()

    sam.annotations[sam.annotations == 255] = 0

    tif_output_name = f'{output_dir}/{filename}'
    geojson_name = f'{output_dir}/{tile_name}.geojson'
    
    array_to_image(sam.annotations, tif_output_name, sam.source)

    sam.tiff_to_geojson(tif_output_name, geojson_name)

    gdf = gpd.read_file(geojson_name)
    
    tolerance = 10
    buffer_distance = 5
    
    gdf['geometry'] = gdf['geometry'].buffer(-buffer_distance).buffer(buffer_distance)
    gdf['geometry'] = gdf['geometry'].simplify(tolerance, preserve_topology=True)
    
    gdf = gdf[gdf['geometry'].area > 10000]
    
    gdf.to_file(geojson_name)

    os.remove(tif_output_name)